In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Checking Total Derivatives

If you want to check the analytic derivatives of your model (or just part of it) against finite difference or complex-step approximations, you can use `check_totals`(). You should always converge your model before calling this method. By default, this method checks the derivatives of all of the driver responses (objectives, constraints) with respect to the design variables, though you can also specify the variables you want to check. Derivatives are computed and compared in an unscaled form by default, but you can optionally request for them to be computed in scaled form using the `ref` and `ref0` that were declared when adding the constraints, objectives, and design variables.

```{Note}
You should probably **not** use this method until you’ve used `check_partials`() to verify the partials for each component in your model. `check_totals`() is a blunt instrument, since it can only tell you that there is a problem, but will not give you much insight into which component or group is causing the problem.

``` 

```{eval-rst}
    .. automethod:: openmdao.core.problem.Problem.check_totals
        :noindex:
```

Please check out the [Complex Step Guidelines](complex-step-guidelines) for more information on how to use it on a model.

## Examples

You can check specific combinations of variables by specifying them manually:

In [2]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src63", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src63`
:::

In [3]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.setup()
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [4]:
# manually specify which derivatives to check
prob.check_totals(of=['obj', 'con1'], wrt=['x', 'z'])

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-----------------
Total Derivatives
-----------------

  Full Model: 'con1' wrt 'x'
     Reverse Magnitude: 9.806145e-01
          Fd Magnitude: 9.806169e-01 (fd:forward)

    Absolute Error (Jrev - Jfd) : 2.407248e-06 *

    Relative Error (Jrev - Jfd) / Jfd : 2.454831e-06 *

    Raw Reverse Derivative (Jrev)
    [[-0.98061448]]

    Raw FD Derivative (Jfd)
    [[-0.98061688]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con1' wrt 'z'
     Reverse Magnitude: 9.641989e+00
          Fd Magnitude: 9.641993e+00 (fd:forward)

    Absolute Error (Jrev - Jfd) : 4.447254e-06 *

    Relative Error (Jrev - Jfd) / Jfd : 4.612380e-07

    Raw Reverse Derivative (Jrev)
    [[-9.61002186 -0.78449158]]

    Raw FD Derivative (Jfd)
    [[-9.61002547 -0.78449417]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'obj' wrt 'x'
     Revers

{('obj', 'x'): {'J_fd': array([[2.98061732]]),
  'J_rev': array([[2.98061391]]),
  'abs error': ErrorTuple(forward=None, reverse=3.4048181052348525e-06, forward_reverse=None),
  'rel error': ErrorTuple(forward=None, reverse=1.1423197752786536e-06, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=None, reverse=2.9806139134842877, fd=2.980617318302393)},
 ('obj', 'z'): {'J_fd': array([[9.61001417, 1.78448793]]),
  'J_rev': array([[9.61001056, 1.78448534]]),
  'abs error': ErrorTuple(forward=None, reverse=4.448706697254583e-06, forward_reverse=None),
  'rel error': ErrorTuple(forward=None, reverse=4.551436600496811e-07, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=None, reverse=9.77428722815846, fd=9.77429125733397)},
 ('con1', 'x'): {'J_fd': array([[-0.98061688]]),
  'J_rev': array([[-0.98061448]]),
  'abs error': ErrorTuple(forward=None, reverse=2.407248432922948e-06, forward_reverse=None),
  'rel error': ErrorTuple(forward=None, reverse=2.4548307050606205e-0

---
Check all the derivatives that the driver will need:

In [5]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [6]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals()

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-----------------
Total Derivatives
-----------------

  Full Model: 'con1' wrt 'x'
     Forward Magnitude: 9.806145e-01
          Fd Magnitude: 9.806169e-01 (fd:forward)

    Absolute Error (Jfor - Jfd) : 2.407248e-06 *

    Relative Error (Jfor - Jfd) / Jfd : 2.454831e-06 *

    Raw Forward Derivative (Jfor)
    [[-0.98061448]]

    Raw FD Derivative (Jfd)
    [[-0.98061688]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con1' wrt 'z'
     Forward Magnitude: 9.641989e+00
          Fd Magnitude: 9.641993e+00 (fd:forward)

    Absolute Error (Jfor - Jfd) : 4.447254e-06 *

    Relative Error (Jfor - Jfd) / Jfd : 4.612380e-07

    Raw Forward Derivative (Jfor)
    [[-9.61002186 -0.78449158]]

    Raw FD Derivative (Jfd)
    [[-9.61002547 -0.78449417]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con2' wrt 'x'
     Forwa

{('obj', 'x'): {'J_fd': array([[2.98061732]]),
  'J_fwd': array([[2.98061391]]),
  'abs error': ErrorTuple(forward=3.4048181047907633e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=1.1423197751296612e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=2.980617318302393)},
 ('obj', 'z'): {'J_fd': array([[9.61001417, 1.78448793]]),
  'J_fwd': array([[9.61001056, 1.78448534]]),
  'abs error': ErrorTuple(forward=4.448706696327325e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=4.5514365995481405e-07, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158462, reverse=None, fd=9.77429125733397)},
 ('con1', 'x'): {'J_fd': array([[-0.98061688]]),
  'J_fwd': array([[-0.98061448]]),
  'abs error': ErrorTuple(forward=2.4072484327009036e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=2.454830704834187e-06, reverse=No

---
Use the driver scaled values during the check:

In [7]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100, ref=100.0, ref0=-100.0)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0, ref=3.0)
prob.model.add_constraint('con2', upper=0.0, ref=20.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [8]:
# check derivatives of all driver vars using the declared scaling
prob.check_totals(driver_scaling=True)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-----------------
Total Derivatives
-----------------

  Full Model: 'con1' wrt 'x'
     Forward Magnitude: 6.537430e+01
          Fd Magnitude: 6.537446e+01 (fd:forward)

    Absolute Error (Jfor - Jfd) : 1.604832e-04 *

    Relative Error (Jfor - Jfd) / Jfd : 2.454831e-06 *

    Raw Forward Derivative (Jfor)
    [[-65.37429835]]

    Raw FD Derivative (Jfd)
    [[-65.37445883]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con1' wrt 'z'
     Forward Magnitude: 3.213996e+00
          Fd Magnitude: 3.213998e+00 (fd:forward)

    Absolute Error (Jfor - Jfd) : 1.482418e-06 *

    Relative Error (Jfor - Jfd) / Jfd : 4.612380e-07

    Raw Forward Derivative (Jfor)
    [[-3.20334062 -0.26149719]]

    Raw FD Derivative (Jfd)
    [[-3.20334182 -0.26149806]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con2' wrt 'x'
     For

{('obj', 'x'): {'J_fd': array([[596.12346366]]),
  'J_fwd': array([[596.1227827]]),
  'abs error': ErrorTuple(forward=0.0006809636209936798, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=1.1423197751892582e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=596.1227826968576, reverse=None, fd=596.1234636604786)},
 ('obj', 'z'): {'J_fd': array([[9.61001417, 1.78448793]]),
  'J_fwd': array([[9.61001056, 1.78448534]]),
  'abs error': ErrorTuple(forward=4.448706696327325e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=4.5514365995481405e-07, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158462, reverse=None, fd=9.77429125733397)},
 ('con1', 'x'): {'J_fd': array([[-65.37445883]]),
  'J_fwd': array([[-65.37429835]]),
  'abs error': ErrorTuple(forward=0.00016048322883932542, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=2.4548307047209705e-06, rever


---
Display the results in a compact format:

In [9]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [10]:
# check derivatives of all obj+constraints w.r.t all design variables
prob.check_totals(compact_print=True)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations
-----------------
Total Derivatives
-----------------

+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+
| of '<variable>' | wrt '<variable>' |   calc mag. |  check mag. |  a(cal-chk) |  r(cal-chk) | error desc         |
+=================+==================+=============+=============+=============+=============+====================+
| 'con1'          | 'x'              |  9.8061e-01 |  9.8062e-01 |  2.4072e-06 |  2.4548e-06 |  >ABS_TOL >REL_TOL |
+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+
| 'con1'          | 'z'              |  9.6420e+00 |  9.6420e+00 |  4.4473e-06 |  4.6124e-07 |  >ABS_TOL          |
+-----------------+------------------+-------------+-------------+-------------+-------------+--------------------+
| 'con2'          | 'x'  

{('obj', 'x'): {'J_fd': array([[2.98061732]]),
  'J_fwd': array([[2.98061391]]),
  'abs error': ErrorTuple(forward=3.4048181047907633e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=1.1423197751296612e-06, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=2.980617318302393)},
 ('obj', 'z'): {'J_fd': array([[9.61001417, 1.78448793]]),
  'J_fwd': array([[9.61001056, 1.78448534]]),
  'abs error': ErrorTuple(forward=4.448706696327325e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=4.5514365995481405e-07, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158462, reverse=None, fd=9.77429125733397)},
 ('con1', 'x'): {'J_fd': array([[-0.98061688]]),
  'J_fwd': array([[-0.98061448]]),
  'abs error': ErrorTuple(forward=2.4072484327009036e-06, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=2.454830704834187e-06, reverse=No

---
Use complex step instead of finite difference for a more accurate check. We also tighten the tolerance on the nonlinear Gauss-Seidel solver so that we get more accurate converged values.

In [11]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup(force_alloc_complex=True)

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [12]:
prob.model.nonlinear_solver.options['atol'] = 1e-15
prob.model.nonlinear_solver.options['rtol'] = 1e-15

# check derivatives with complex step
prob.check_totals(method='cs')

NL: NLBGS Converged in 7 iterations
NL: NLBGS Converged in 7 iterations
NL: NLBGS Converged in 7 iterations
-----------------
Total Derivatives
-----------------

  Full Model: 'con1' wrt 'x'
     Forward Magnitude: 9.806145e-01
          Fd Magnitude: 9.806145e-01 (cs:None)

    Absolute Error (Jfor - Jfd) : 2.139111e-11

    Relative Error (Jfor - Jfd) / Jfd : 2.181399e-11

    Raw Forward Derivative (Jfor)
    [[-0.98061448]]

    Raw CS Derivative (Jfd)
    [[-0.98061448]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con1' wrt 'z'
     Forward Magnitude: 9.641989e+00
          Fd Magnitude: 9.641989e+00 (cs:None)

    Absolute Error (Jfor - Jfd) : 9.613199e-09

    Relative Error (Jfor - Jfd) / Jfd : 9.970141e-10

    Raw Forward Derivative (Jfor)
    [[-9.61002186 -0.78449158]]

    Raw CS Derivative (Jfd)
    [[-9.61002187 -0.78449158]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  Full Model: 'con2' wrt 'x'
     Forward Magnitude

{('obj', 'x'): {'J_fd': array([[2.98061392]]),
  'J_fwd': array([[2.98061391]]),
  'abs error': ErrorTuple(forward=9.598639216790161e-09, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=3.2203564314233746e-09, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=2.9806139230829274)},
 ('obj', 'z'): {'J_fd': array([[9.61001057, 1.78448534]]),
  'J_fwd': array([[9.61001056, 1.78448534]]),
  'abs error': ErrorTuple(forward=9.613365873086588e-09, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=9.835362558295142e-10, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=9.774287228158462, reverse=None, fd=9.774287237615534)},
 ('con1', 'x'): {'J_fd': array([[-0.98061448]]),
  'J_fwd': array([[-0.98061448]]),
  'abs error': ErrorTuple(forward=2.139111110466274e-11, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=2.18139866841579e-11, reverse=None

Note that you need to set the argument `force_alloc_complex` to True when you call `setup`. This makes sure that the vectors allocate space to pass complex numbers. All systems also have an attribute `force_alloc_complex` that can be queried during `setup` in case you need to set up your `Component` or `Group` differently to accomodate complex step.

In [13]:
import openmdao.api as om

class SimpleComp(om.ExplicitComponent):

    def setup(self):
        self.add_input('x', val=1.0)
        self.add_output('y', val=1.0)

        self.declare_partials(of='y', wrt='x')

        if self._force_alloc_complex:
            print("Vectors allocated for complex step.")

    def compute(self, inputs, outputs):
        outputs['y'] = 3.0*inputs['x']

    def compute_partials(self, inputs, partials):
        partials['y', 'x'] = 3.

prob = om.Problem()
prob.model.add_subsystem('comp', SimpleComp())

prob.model.add_design_var('comp.x', lower=-100, upper=100)
prob.model.add_objective('comp.y')

prob.setup(force_alloc_complex=True)

prob.run_model()

prob.check_totals(method='cs')

Vectors allocated for complex step.
-----------------
Total Derivatives
-----------------

  Full Model: 'comp.y' wrt 'comp.x'
     Forward Magnitude: 3.000000e+00
          Fd Magnitude: 3.000000e+00 (cs:None)

    Absolute Error (Jfor - Jfd) : 0.000000e+00

    Relative Error (Jfor - Jfd) / Jfd : 0.000000e+00

    Raw Forward Derivative (Jfor)
    [[3.]]

    Raw CS Derivative (Jfd)
    [[3.]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -


{('comp.y', 'comp.x'): {'J_fd': array([[3.]]),
  'J_fwd': array([[3.]]),
  'abs error': ErrorTuple(forward=0.0, reverse=None, forward_reverse=None),
  'rel error': ErrorTuple(forward=0.0, reverse=None, forward_reverse=None),
  'magnitude': MagnitudeTuple(forward=3.0, reverse=None, fd=3.0)}}


---
Turn off standard output and just view the derivatives in the return:

In [14]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [15]:
# check derivatives of all obj+constraints w.r.t all design variables
totals = prob.check_totals(out_stream=None)

NL: NLBGS Converged in 4 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 5 iterations


In [16]:
print(totals)

{('obj', 'x'): {'J_fd': array([[2.98061732]]), 'J_fwd': array([[2.98061391]]), 'abs error': ErrorTuple(forward=3.4048181047907633e-06, reverse=None, forward_reverse=None), 'rel error': ErrorTuple(forward=1.1423197751296612e-06, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=2.980617318302393)}, ('obj', 'z'): {'J_fd': array([[9.61001417, 1.78448793]]), 'J_fwd': array([[9.61001056, 1.78448534]]), 'abs error': ErrorTuple(forward=4.448706696327325e-06, reverse=None, forward_reverse=None), 'rel error': ErrorTuple(forward=4.5514365995481405e-07, reverse=None, forward_reverse=None), 'magnitude': MagnitudeTuple(forward=9.774287228158462, reverse=None, fd=9.77429125733397)}, ('con1', 'x'): {'J_fd': array([[-0.98061688]]), 'J_fwd': array([[-0.98061448]]), 'abs error': ErrorTuple(forward=2.4072484327009036e-06, reverse=None, forward_reverse=None), 'rel error': ErrorTuple(forward=2.454830704834187e-06, reverse=None, forward_reverse=None


---
Evaluate the total derivatives using multiple FD step sizes:

In [17]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem()
prob.model = SellarDerivatives()
prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.linear_solver = om.ScipyKrylov()

prob.model.add_design_var('x', lower=-100, upper=100)
prob.model.add_design_var('z', lower=-100, upper=100)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0.0)
prob.model.add_constraint('con2', upper=0.0)

prob.setup()

# We don't call run_driver() here because we don't
# actually want the optimizer to run
prob.run_model()

NL: NLBGS Converged in 8 iterations


In [18]:
# check derivatives of all obj+constraints w.r.t all design variables for FD steps of 1e-6 and 1e-7
prob.check_totals(step=[1.e7, 1.e8])

NL: NLBGS Converged in 2 iterations
NL: NLBGS Converged in 3 iterations
NL: NLBGS Converged in 5 iterations
NL: NLBGS Converged in 2 iterations
NL: NLBGS Converged in 3 iterations
NL: NLBGS Converged in 4 iterations
-----------------
Total Derivatives
-----------------

  Full Model: 'con1' wrt 'x'
     Forward Magnitude: 9.806145e-01
          Fd Magnitude: 9.999369e-01 (fd:forward, step=10000000.0)
          Fd Magnitude: 9.999800e-01 (fd:forward, step=100000000.0)

    Absolute Error (Jfor - Jfd), step=10000000.0 : 1.932238e-02 *
    Absolute Error (Jfor - Jfd), step=100000000.0 : 1.936553e-02 *

    Relative Error (Jfor - Jfd) / Jfd, step=10000000.0 : 1.932360e-02 *
    Relative Error (Jfor - Jfd) / Jfd, step=100000000.0 : 1.936592e-02 *

    Raw Forward Derivative (Jfor)
    [[-0.98061448]]

    Raw FD Derivative (Jfd), step=10000000.0
    [[-0.99993686]]

    Raw FD Derivative (Jfd), step=100000000.0
    [[-0.99998001]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

{('obj',
  'x'): {'J_fd': [array([[10000002.99993685]]),
   array([[1.00000003e+08]])], 'steps': [10000000.0,
   100000000.0], 'J_fwd': array([[2.98061391]]), 'abs error': [ErrorTuple(forward=10000000.019322941, reverse=None, forward_reverse=None),
   ErrorTuple(forward=100000000.01936609, reverse=None, forward_reverse=None)], 'rel error': [ErrorTuple(forward=0.999999701938698, reverse=None, forward_reverse=None),
   ErrorTuple(forward=0.9999999701938618, reverse=None, forward_reverse=None)], 'magnitude': [MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=10000002.999936854),
   MagnitudeTuple(forward=2.980613913484288, reverse=None, fd=100000002.99998)]},
 ('obj',
  'z'): {'J_fd': [array([[1.00000096e+07, 1.79994353e+00]]),
   array([[1.00000010e+08, 1.79998212e+00]])], 'steps': [10000000.0,
   100000000.0], 'J_fwd': array([[9.61001056, 1.78448534]]), 'abs error': [ErrorTuple(forward=9999999.989989448, reverse=None, forward_reverse=None),
   ErrorTuple(forward=99999999.989989